In [1]:
import pandas as pd
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
data_train = pd.read_excel('/content/drive/MyDrive/Asssignment4_Data.xlsx',sheet_name='Train')
data_test =  pd.read_excel('/content/drive/MyDrive/Asssignment4_Data.xlsx',sheet_name='Test')

In [3]:
X_train = data_train.drop(['House ID','Construction type'],axis =1)
X_test = data_test.drop(['House ID','Construction type'],axis= 1)

y_train = data_train['Construction type']
y_test = data_test['Construction type']

In [4]:
class NaiveBayesClassifier():
    def calc_prior(self, features, target):
        self.prior = (features.groupby(target).apply(lambda x: len(x)) / self.rows).to_numpy()
        return self.prior
    
    def calc_statistics(self, features, target):
        self.mean = features.groupby(target).apply(np.mean).to_numpy()
        self.var = features.groupby(target).apply(np.var).to_numpy()
        return self.mean, self.var
    
    def gaussian_density(self, class_idx, x):     
        mean = self.mean[class_idx]
        var = self.var[class_idx]
        numerator = np.exp((-1/2)*((x-mean)**2) / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        prob = numerator / denominator
        return prob
    
    def calc_posterior(self, x):
        posteriors = []
        for i in range(self.count):
            prior = np.log(self.prior[i]) 
            conditional = np.sum(np.log(self.gaussian_density(i, x))) 
            posterior = prior + conditional
            posteriors.append(posterior)
        return self.classes[np.argmax(posteriors)]
     

    def fit(self, features, target):
        self.classes = np.unique(target)
        self.count = len(self.classes)
        self.feature_nums = features.shape[1]
        self.rows = features.shape[0]
        
        self.calc_statistics(features, target)
        self.calc_prior(features, target)
        
    def predict(self, features):
        preds = [self.calc_posterior(f) for f in features.to_numpy()]
        return preds

    def accuracy(self, y_test, y_pred):
        accuracy = np.sum(y_test == y_pred) / len(y_test)
        return accuracy

In [5]:
model = NaiveBayesClassifier()
model.fit(X_train, y_train)

In [6]:
y_preds = model.predict(X_test)

In [7]:
y_train

0     Apartment
1         House
2         Condo
3     Apartment
4     Apartment
5         Condo
6         Condo
7         House
8         Condo
9     Apartment
10        House
11        House
12        House
13        Condo
14    Apartment
15        House
16    Apartment
17        House
18        Condo
19    Apartment
Name: Construction type, dtype: object

In [8]:
model.accuracy(y_test, y_preds)

0.4